In [ ]:
from notebook_prelude import *
import experiments

In [ ]:
data = collections.defaultdict(lambda: [])
cmap_cache_files = dataset_helper.get_all_cached_graph_datasets(graph_type=TYPE_CONCEPT_MAP)
for file in helper.log_progress(cmap_cache_files):
    dataset = filename_utils.get_dataset_from_filename(file)
    X, Y = dataset_helper.get_dataset_cached(file)
    X = graph_helper.get_graphs_only(X)
    
    all_labels = set(graph_helper.get_all_node_labels(X))
    data['dataset'] += [dataset] * len(all_labels)
    data['labels'] += [str(x) for x in all_labels]

df = pd.DataFrame(data)
df['num_words'] = df['labels'].str.split().apply(len)
df = df.set_index(['dataset', 'labels'])

In [ ]:
num_datasets = len(df.reset_index().dataset.unique())
fig, axes = plt.subplots(ncols=2, nrows=int(np.ceil(num_datasets / 2)), sharex=False)

for ax, (dataset, df_) in zip(axes.flatten()[:num_datasets], df.groupby('dataset')):
    print(dataset)
    df_.reset_index().set_index('labels').num_words.plot(kind='hist', bins=50, ax=ax, title=dataset)
    ax.grid('off')

fig.tight_layout()

In [ ]:
df.groupby('dataset').num_words.describe()

In [ ]:
from preprocessing import preprocessing

df_ = df.reset_index()
df_[(df_.dataset=='ng20') & (df_.num_words > 10)]

df_['label_clean'] = df_['labels'].apply(preprocessing.preprocess)

In [ ]:
df_[(df_.dataset=='ng20') & (df_.num_words > 10)]